In [ ]:
import os
import numpy as np
import pandas as pd
#os.getcwd()
import folium
from folium import plugins
import geopandas
import geojson
import matplotlib.cm as cm
import webbrowser
import glob

In [ ]:
dir_gpx_original  = 'data/gpx_original' 
dir_gpx_processed = 'data/gpx_processed' 
dir_geojson       = 'data/geojson'

dir_work = '/home/craigmatthewsmith/gps_tracks'
os.chdir(dir_work)
#os.getcwd()

#gpx_file_temp = os.path.join(dir_work, 'data_input_gpx/Afternoon_Run55.gpx')
#print(os.path.isfile(gpx_file_temp))


In [ ]:
# functions
def rgb2hex(c):
    hexc = '#%02x%02x%02x'%(int(c[0]*255), int(c[1]*255), int(c[2]*255))
    return(hexc)


In [ ]:

use_RDP  = True
epsilon  = 1.0 # rdp thinning 
#dist_min_aggregate_points = 1.0 
dist_min_aggregate_points = 3.0 
#dist_min_aggregate_points = 10.0 

#  1.0, reduced points from 75752 to 65536, 2.5 M master.geojson file size 
# 10.0, reduced points from 75752 to 19330, 2.5 M master.geojson file size

dist_max_between_points_to_make_line = 100.0 # dont plot lines this far away

# thin2 deprecated only
dist_min_adjacent_points_to_line = 30.0 


In [ ]:
#geojson_file = os.path.join(dir_work, 'master_thin.geojson')
geojson_file = 'master_thin_min_'+str(int(dist_min_aggregate_points))+'_max_'+str(int(dist_max_between_points_to_make_line))+'.geojson'

print(os.path.isfile(geojson_file))

# read geojson file
with open(geojson_file, 'r') as file:
    geojson_data = geojson.load(file)

features_tracks  = []
features_n_times = []

for feature in geojson_data['features']:
    line = geojson.LineString(feature['geometry']['coordinates'])
    n_times = feature['properties']['n_times']
    features_tracks.append(geojson.Feature(geometry=line))
    features_n_times.append(geojson.Feature(geometry=line, properties={'n_times': n_times}))
    
geojson_data_track = geojson.FeatureCollection(features_tracks)
geojson_data_n_times = geojson.FeatureCollection(features_n_times)
    

In [ ]:
cmin_n_times = min(feature['properties']['n_times'] for feature in geojson_data['features'])
cmax_n_times = max(feature['properties']['n_times'] for feature in geojson_data['features'])

print(cmin_n_times)
print(cmax_n_times)

# create new GeoJson objects to reduce GeoJSON data sent to Folium map as layer
style_track = lambda x: {'color': '#FC4C02', 'weight': 5} # show some color...

style_n_times = lambda x: {'color': rgb2hex(cmap((x['properties']['n_times']-cmin_n_times)/(cmax_n_times-cmin_n_times))), 'weight': 5} # cmap needs normalized data
tooltip_n_times = folium.features.GeoJsonTooltip(fields=['n_times'], aliases=['n_times'])



In [ ]:
# set up Folium map
#fmap = folium.Map(tiles = None, prefer_canvas=True, disable_3d=True)
#fmap = folium.Map(tiles='Stamen Terrain', prefer_canvas=True, disable_3d=True)
fmap = folium.Map(tiles='Stamen Terrain', location=[37.862606, -121.978372], zoom_start=10) 
folium.TileLayer(tiles = 'OpenStreetMap', name='OpenStreetMap', show=False).add_to(fmap)
folium.TileLayer(tiles = 'Stamen Terrain', name='Terrain Map', show=True).add_to(fmap)
cmap = cm.get_cmap('jet') # matplotlib colormap


In [ ]:
print('appending features to map ')

folium.GeoJson(geojson_data_track, style_function=style_track, name='track', show=True, smooth_factor=3.0).add_to(fmap)
folium.GeoJson(geojson_data_n_times, style_function=style_n_times, tooltip=tooltip_n_times, name='n_times', show=False, smooth_factor=3.0).add_to(fmap)
        
fmap

In [ ]:
# add layer control widget
folium.LayerControl(collapsed=False).add_to(fmap)

# save map to html file
fmap.fit_bounds(fmap.get_bounds())

#html_file = os.path.join(dir_work, 'heatmap.html')
#html_file = 'heatmap_'+str(int(dist_min))+'_max_'+str(int(dist_max))+'.html'
html_file = 'heatmap_'+str(int(dist_min_aggregate_points))+'_max_'+str(int(dist_max_between_points_to_make_line))+'.html'


geojson_file = 'master_thin_min_'+str(int())+'_max_'+str(int())+'.geojson'


if os.path.isfile(html_file):
    os.system('rm -f '+html_file)
fmap.save(html_file)
# open html file in default browser
webbrowser.open(html_file, new=2, autoraise=True)
